In [571]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [572]:
df=pd.read_csv("superstore.csv")
tempdf=df

Removing Miscellaneous(Unwanted) columns:

In [573]:
print(tempdf.columns)
tempdf.pop("Unnamed: 0")
tempdf.pop("记录数")
tempdf.pop("Row.ID")
tempdf.pop('Region')

Index(['Unnamed: 0', 'Category', 'City', 'Country', 'Customer.ID',
       'Customer.Name', 'Discount', 'Market', '记录数', 'Order.Date', 'Order.ID',
       'Order.Priority', 'Product.ID', 'Product.Name', 'Profit', 'Quantity',
       'Region', 'Row.ID', 'Sales', 'Segment', 'Ship.Date', 'Ship.Mode',
       'Shipping.Cost', 'State', 'Sub.Category', 'Year', 'Market2', 'weeknum'],
      dtype='object')


0        West
1        West
2        West
3        West
4        West
         ... 
51285    West
51286    West
51287    West
51288    West
51289    West
Name: Region, Length: 51290, dtype: object

Rearraging the columns for convenience

In [574]:
columns = [
    # Order Details
    "Order.ID", "Order.Date", "weeknum", "Year", "Ship.Date", "Order.Priority", "Ship.Mode",    
    # Customer Details
    "Customer.ID", "Customer.Name", "Segment",    
    # Location Details
    "City", "State", "Country",   
    # Product Details
    "Product.ID", "Product.Name", "Category", "Sub.Category",    
    # Sales and Financials
    "Quantity", "Sales", "Discount", "Profit",    
    # Market Details
    "Market", "Market2"
]
tempdf=tempdf[columns]

Cleaning the dataset:

In [575]:
for i in tempdf.columns:
    if(tempdf[i].dtypes=='object'):
        tempdf[i]=tempdf[i].astype("string")
tempdf['Order.Date']=pd.to_datetime(tempdf['Order.Date'])
tempdf['Ship.Date']=pd.to_datetime(tempdf['Ship.Date'])
tempdf['Year']=pd.to_numeric(tempdf['Year'])

#setting the dataframe index to show the row number
tempdf=tempdf.sort_values(by='Order.Date', ascending=False)
tempdf.reset_index()
tempdf.set_index(np.arange(0,len(tempdf)), inplace=True)

C:\Users\mooha\AppData\Local\Temp\ipykernel_26868\2788352954.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf[i]=tempdf[i].astype("string")
C:\Users\mooha\AppData\Local\Temp\ipykernel_26868\2788352954.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf['Order.Date']=pd.to_datetime(tempdf['Order.Date'])
C:\Users\mooha\AppData\Local\Temp\ipykernel_26868\2788352954.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [576]:
print(len(tempdf))
tempdf = tempdf[tempdf['Ship.Date'] >= tempdf['Order.Date']]
print(df.duplicated().sum())
print(len(tempdf))

51290
0
50715


In [577]:
#filling up the Market and Market2 columns
M1=tempdf["Market"].dropna().unique().tolist()
M2=tempdf["Market2"].dropna().unique().tolist()
M2.append("North America")
#print(M1,"\n", M2)

for i in range(len(M1)): 
    tempdf.loc[tempdf["Market"].isna() & (tempdf["Market2"]==M2[i]), "Market"] = M1[i]
    tempdf.loc[tempdf["Market2"].isna() & (tempdf["Market"]==M1[i]), "Market2"] = M2[i]

Cleaning Location related information

In [578]:
c=pd.read_csv("cities.csv")
city_state_country=c.loc[:,["name", "state_name", "country_name"]]

In [ ]:
tempdf.loc[:,['City','State','Country']].nunique

print(len(tempdf[tempdf["Country"].isna()]))
print(len(tempdf[tempdf["State"].isna()]))
print(len(tempdf[tempdf["City"].isna()]))

# Creating a mapping dictionary 
state_to_country = dict(zip(city_state_country['state_name'], city_state_country['country_name']))
city_to_country = dict(zip(city_state_country['name'], city_state_country['country_name']))
city_to_state = dict(zip(city_state_country['state_name'], city_state_country['name']))

# mapping using np.where
tempdf['Country'] = np.where(tempdf['Country'].isna(), tempdf['State'].map(state_to_country), tempdf['Country'])
#tempdf['Country'] = np.where(tempdf['Country'].isna(), tempdf['City'].map(city_to_country), tempdf['Country'])

tempdf['State'] = np.where(tempdf['State'].isna(), tempdf['City'].map(city_to_state), tempdf['State'])

print(len(tempdf[tempdf["Country"].isna()]))
print(len(tempdf[tempdf["State"].isna()]))
print(len(tempdf[tempdf["City"].isna()]))

<bound method DataFrame.nunique of               City             State      Country
0           Manila  National Capital  Philippines
1           Agadir  Souss-Massa-Draâ      Morocco
2          Managua           Managua    Nicaragua
3        São Paulo         São Paulo       Brazil
4        Eindhoven     North Brabant  Netherlands
...            ...               ...          ...
51080  Wagga Wagga   New South Wales    Australia
51081  Wagga Wagga   New South Wales    Australia
51082    Stockholm         Stockholm       Sweden
51083  Wagga Wagga   New South Wales    Australia
51084  Constantine       Constantine      Algeria

[50715 rows x 3 columns]>

In [580]:
tempdf['Year'] = np.where(tempdf['Year'].isna(), pd.DatetimeIndex(tempdf['Order.Date']).year, tempdf['Year'])

In [581]:
before=len(tempdf)
#tempdf=tempdf.dropna()
after=len(tempdf)
((before-after)/before)*100

tempdf.isna().sum()

Order.ID          301
Order.Date          0
weeknum             0
Year                0
Ship.Date           0
Order.Priority    133
Ship.Mode         463
Customer.ID       224
Customer.Name     423
Segment           424
City              244
State             412
Country           403
Product.ID        285
Product.Name       29
Category          116
Sub.Category      275
Quantity          147
Sales             177
Discount          459
Profit            323
Market              0
Market2             0
dtype: int64

In [582]:
freq_table = pd.crosstab(tempdf['Year'], tempdf['Segment'], margins=True, dropna=True, normalize=True) 
round(freq_table*100, 2)
#(freq_table/len(tempdf.dropna()))*100

Segment,Consumer,Corporate,Home Office,All
Year,,,,
2011.0,9.16,5.35,3.04,17.54
2012.0,11.27,6.37,3.72,21.37
2013.0,13.87,8.19,4.87,26.93
2014.0,17.43,10.14,6.59,34.16
All,51.73,30.06,18.22,100.00


In [583]:
#plt.bar(tempdf['Year'], tempdf['Sales'])

In [584]:
avgdate=tempdf['Ship.Date']-tempdf['Order.Date']
tempdf['avgdate']=avgdate